In [1]:
%pylab inline
import torch
import torchvision
from PIL import Image
import json
class_idx = json.load(open("imagenet_class_index.json"))
I1 = Image.open('dog.jpg')
I2 = Image.open('cat.jpg')

Populating the interactive namespace from numpy and matplotlib


In [2]:
model = torchvision.models.resnext101_32x8d(pretrained=True, progress=False) # using ResNet
# model = torchvision.models.mobilenet_v2(pretrained=True, progress=False)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt101_32X8D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt101_32X8D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth" to /root/.cache/torch/hub/checkpoints/resnext101_32x8d-8ba56ff5.pth


In [4]:
transform = torchvision.transforms.Compose([ # transform images to be valid for model
    torchvision.transforms.Resize(224), # Scale to 224
    torchvision.transforms.CenterCrop(224), # Crop to center
    torchvision.transforms.ToTensor(), # Make tensor out of image
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]) # Normalize, copy mean and std dev values from online (normal values)

model.eval() # Need to set model to evaluation mode so that training augmentations are not turned on

p = model(transform(I1)[None])[0] # Model predicts label for I1 (dog)
print( ' , '.join([class_idx[str(int(i))][1] for i in p.argsort(descending=True)[:5]]) ) # Print out prob, first 5 labels
p = model(transform(I2)[None])[0] # Model predicts label for I2 (cat)
print( ' , '.join([class_idx[str(int(i))][1] for i in p.argsort(descending=True)[:5]]) ) # Print out top 5 labels

wire-haired_fox_terrier , toy_terrier , Brabancon_griffon , standard_schnauzer , Lakeland_terrier
tiger_cat , tabby , Egyptian_cat , lynx , tiger


In [5]:
model.fc = torch.nn.Linear(2048, 10) # Set it to a 10 class classifier

In [7]:
model(transform(I1)[None]) # Now prints out 10 outputs, don't correspond to anything right now because we haven't trained this mdoel

tensor([[-0.0619,  0.3418, -0.0826,  0.1336,  0.6247,  0.1160, -0.5546,  0.0695,
         -0.4975, -0.2046]], grad_fn=<AddmmBackward0>)